# About

This notebook extracts spectral and date features of NAIP images at random points within polygons in the 'polygons_form_naip_images' folder. These polygons are known ice plant and non ice plant locations within a specific NAIP image. 


Once the aoi and years are specified, the notebook samples first polygons labeled as ice plant locations and then polygons labeled as non-ice plant locations. Two methods for sampling polygons are implemented in the `extracting_points_from_polygons,` and both are used in this notebook. The first one, `naip_sample_proportion`, samples a fixed fraction of the points in each polygon. The second one, `naip_sample_sliding`, samples a fixed fraction of the points in each polygon up to a maximum number of points. Polygons vary greatly in size, so simply sampling a fraction of the points in each polygon would result in an over-sampling of bigger polygons (most often those corresponding to non-ice plant locations), which in turn would unbalance the training set towards one label. The parameters used in this notebook were determined to obtain a final training set with a 3:1 proportion of non-ice plant to ice plant points. 

**NOTEBOOK VARIABLES:**

- `aois` (array): These are the areas of interest from which the polygons we want to sample were collected. Must be a subset of: ['campus_lagoon','carpinteria','gaviota','point_conception']. 

- `years` (array): can be any subset from [2012, 2014, 2016, 2018, 2020]. If aoi = 'point_conception' then 2016 will not be included in the outcome since there are not points to NAIP images to sample from on that year. 

- `sample_fraction` (float in (0,1]): fraction of points to sample from each polygon

- `max_sample` (int): maximum number of points to sample from a polygon

**OUTPUT:**

The output is a dataframe of points with the following features:

- geometry: coordinates of point *p* in the NAIP image's CRS
- naip_id: itemid of the NAIP from which *p* was sampled from
- polygon_id: id of the polygon from which *p* was sampled from
- iceplant: whether point *p* corresponds to a confirmed iceplant location or a confirmed non-iceplant location (0 = non-iceplant, 1 = iceplant)
- r, g, b, nir: Red, Green, Blue and NIR bands values of NAIP scene with naip_id at at cooridnates of point *p*
- ndvi: computed for each point using the Red and NIR bands
- year, month, day_in_year: year, month and day of the year when the NAIP image was collected
- aoi: name of the area of interest where the points were sampled from


The dataframe is then saved in the 'temp' folder as a csv file.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import geopandas as gpd

import extracting_points_from_polygons as pp

# Specify notebook variables

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

aois = ['campus_lagoon','carpinteria','gaviota','point_conception']

# years = array of years, can be any subset from [2012, 2014, 2016, 2018, 2020]
years = [2012, 2014, 2016, 2018, 2020]

# sample 90% of pts in each polygon
sample_fraction = 0.9

# maximum number of pts to sample in a polygon
max_sample = 1000

# ***************************************************
# ***************************************************

# Sample points

In [3]:
for aoi in aois:
    for year in years:
        
        if ('point_conception' != aoi) or (year != 2016):  #there's no data for Point Conception on 2016

            # open polygons
            fp = pp.path_to_polygons(aoi,year)
            polys = gpd.read_file(fp)
            
            # select iceplant polygons and sample sample_fraction of pts in each polygon 
            polys_ice = polys.loc[polys.iceplant==1]
            polys_ice.reset_index(inplace=True, drop=True)

            pts_ice = pp.naip_sample_proportion_no_warnings(polys_ice, 
                                                            polys.naip_id[0], 
                                                            sample_fraction)  
            
            # select non-iceplant polygons and sample sample_fraction of pts in each polygon,  but at most max_sample points 
            polys_nonice = polys.loc[polys.iceplant==0]
            polys_nonice.reset_index(inplace=True, drop=True)

            pts_nonice = pp.naip_sample_sliding_no_warnings(polys_nonice, polys.naip_id[0], 
                                                            sample_fraction, 
                                                            max_sample)
            # assemble into single dataframe
            pts = pd.concat([pts_ice,pts_nonice])

            # add name of aoi
            pts['aoi'] = aoi
            
            # add ndvi as feature
            pts['ndvi']=(pts.nir.astype('int16') - pts.r.astype('int16'))/(pts.nir.astype('int16') + pts.r.astype('int16'))

            # Save points as csv
            fp = os.path.join(os.getcwd(), 
                              'temp', 
                              aoi+'_points_'+str(year)+'.csv')
            pts.to_csv(fp)
            
            
            # print sample information
            print('************ '+aoi+ ' ' +str(year)+' ************')
            print('** # ice plant pts sampled')
            print(pts_ice.shape[0], '\n')
            
            print('** # non ice plant pts sampled')
            print(pts_nonice.shape[0], '\n')
            #pts_nonice.polygon_id.value_counts()
            
            print('*** proportions')
            pp.iceplant_proportions(pts.iceplant)

            print( '---------------------------------------')

************ campus_lagoon 2012************
** # ice plant pts sampled
4013 

** # non ice plant pts sampled
14366 

*** proportions
no-iceplant:iceplant ratio     3.6 :1
          counts  percentage
iceplant                    
0          14366       78.17
1           4013       21.83

---------------------------------------
************ campus_lagoon 2014************
** # ice plant pts sampled
2413 

** # non ice plant pts sampled
11024 

*** proportions
no-iceplant:iceplant ratio     4.6 :1
          counts  percentage
iceplant                    
0          11024       82.04
1           2413       17.96

---------------------------------------
************ campus_lagoon 2016************
** # ice plant pts sampled
6714 

** # non ice plant pts sampled
13569 

*** proportions
no-iceplant:iceplant ratio     2.0 :1
          counts  percentage
iceplant                    
0          13569        66.9
1           6714        33.1

---------------------------------------
************ cam